# Limpeza e EDA dos dados recebidos


## Imports

In [1]:
import pandas as pd
import numpy as np

## Leitura e carregamento dos dados

In [2]:
df = pd.read_csv('../data/vendas_jittered.csv')
df.head()

,Data,Produto,Quantidade,ValorMedio,Total,Func,Ndoc,Tipo,CST,NCM,Codigo,Categoria
0,2015-01-02,OLEO SELENIA SPRINTA 4T 20W-50,1.0,11.70,11.70,4,24039,C,60,27101932,913,OLEO
1,2015-01-02,PNEU TRAS. BIZ LEVORIN DAKA 80/100X14,1.0,109.25,109.25,3,24049,C,60,40114000,829,PNEU
2,2015-01-02,CAMARA AR CG/TITAN LEVORIN (SA 18),1.0,24.61,24.61,4,24039,C,60,40139000,442,CAMARA
3,2015-01-02,CAMARA AR TRAS. BROS/XLR 250 PIRELLI,1.0,37.75,37.75,4,24043,C,60,40139000,1376,CAMARA
4,2015-01-02,CABO EMB. KS/ES 2002/08 FAN IMP/CG-83,1.0,10.39,10.39,3,24037,C,60,84831030,2146,CABO


## Análise

### Início da análise estrutural dos dados

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255895 entries, 0 to 255894
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Data        255895 non-null  object 
 1   Produto     255895 non-null  object 
 2   Quantidade  255895 non-null  float64
 3   ValorMedio  255895 non-null  float64
 4   Total       255895 non-null  float64
 5   Func        255895 non-null  int64  
 6   Ndoc        255895 non-null  int64  
 7   Tipo        255895 non-null  object 
 8   CST         255895 non-null  int64  
 9   NCM         255895 non-null  int64  
 10  Codigo      255895 non-null  int64  
 11  Categoria   255895 non-null  object 
dtypes: float64(3), int64(5), object(4)
memory usage: 23.4+ MB


In [16]:
df.describe()

,Quantidade,ValorMedio,Total,Func,Ndoc,CST,NCM,Codigo
count,255895.000000,255895.000000,255895.000000,255895.000000,255895.000000,255895.000000,2.558950e+05,255895.000000
mean,1.278110,21.954933,23.406216,3.636097,67053.554774,58.289619,6.895641e+07,3055.969562
std,1.095199,35.574053,38.087845,1.009804,51968.250656,10.749343,2.257513e+07,2518.062502
min,1.000000,0.100000,0.100000,0.000000,1.000000,0.000000,2.710193e+07,1.000000
25%,1.000000,3.890000,4.660000,3.000000,17108.000000,60.000000,4.016930e+07,787.000000
50%,1.000000,11.220000,12.300000,4.000000,60796.000000,60.000000,8.421310e+07,2635.000000
75%,1.000000,22.600000,23.660000,4.000000,111154.500000,60.000000,8.714100e+07,5060.000000
max,178.000000,762.310000,1699.080000,9.000000,166092.000000,260.000000,9.613900e+07,8971.000000


In [15]:
df.describe(include='object')

,Data,Produto,Tipo,Categoria
count,255895,255895,255895,255895
unique,3054,6582,2,223
top,2016-09-02,OLEO SELENIA SPRINTA 4T 20W-50,P,OLEO
freq,286,8937,163009,24361


### Conversão dos dados necessários

Podemos ver que temos que categorizar os Funcinoários, converter Data para datetime e mudar quantidade para valores inteiros, não se vende "valores quebrados" de peças.

In [18]:
df['Func'] = df['Func'].astype('category')
df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')

In [19]:
df.head()

,Data,Produto,Quantidade,ValorMedio,Total,Func,Ndoc,Tipo,CST,NCM,Codigo,Categoria
0,2015-01-02,OLEO SELENIA SPRINTA 4T 20W-50,1.0,11.70,11.70,4,24039,C,60,27101932,913,OLEO
1,2015-01-02,PNEU TRAS. BIZ LEVORIN DAKA 80/100X14,1.0,109.25,109.25,3,24049,C,60,40114000,829,PNEU
2,2015-01-02,CAMARA AR CG/TITAN LEVORIN (SA 18),1.0,24.61,24.61,4,24039,C,60,40139000,442,CAMARA
3,2015-01-02,CAMARA AR TRAS. BROS/XLR 250 PIRELLI,1.0,37.75,37.75,4,24043,C,60,40139000,1376,CAMARA
4,2015-01-02,CABO EMB. KS/ES 2002/08 FAN IMP/CG-83,1.0,10.39,10.39,3,24037,C,60,84831030,2146,CABO


In [35]:
df['Quantidade'] = df['Quantidade'].astype(int)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255895 entries, 0 to 255894
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Data        255895 non-null  datetime64[ns]
 1   Produto     255895 non-null  object        
 2   Quantidade  255895 non-null  int64         
 3   ValorMedio  255895 non-null  float64       
 4   Total       255895 non-null  float64       
 5   Func        255895 non-null  category      
 6   Ndoc        255895 non-null  int64         
 7   Tipo        255895 non-null  object        
 8   CST         255895 non-null  int64         
 9   NCM         255895 non-null  int64         
 10  Codigo      255895 non-null  int64         
 11  Categoria   255895 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(2), int64(5), object(3)
memory usage: 21.7+ MB


### Contagem dos valores nulos

In [20]:
df.isna().sum()

Data          0
Produto       0
Quantidade    0
ValorMedio    0
Total         0
Func          0
Ndoc          0
Tipo          0
CST           0
NCM           0
Codigo        0
Categoria     0
dtype: int64

ok!
---